# 📘 Agentic RAG Core Notebook
This notebook implements a LangGraph-based RAG pipeline using modular agents.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Set base path for project
import sys
base_path = '/content/drive/MyDrive/PharmaAGen'
sys.path.append("/content/drive/MyDrive/PharmaAGen/src")

In [7]:
for p in sys.path:
    print(p)

/content
/env/python
/usr/lib/python311.zip
/usr/lib/python3.11
/usr/lib/python3.11/lib-dynload

/usr/local/lib/python3.11/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.11/dist-packages/IPython/extensions
/root/.ipython
/content/drive/MyDrive/PharmaAGen/src
/content/drive/MyDrive/PharmaAGen/src


In [2]:
# Install dependencies
!pip install langgraph faiss-cpu sentence-transformers pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6

In [32]:
!ls /content/drive/MyDrive/PharmaAGen/src/utils/

ls: cannot access '/content/drive/MyDrive/PharmaAGen/src/utils/': No such file or directory


In [6]:
# Imports and setup
import sys
# Removed redundant sys.path.append as base_path is now set correctly
sys.path.append(base_path + '/src')

In [8]:
from utils.pdf_utils import extract_text_from_pdf, clean_text, chunk_text
from agents.embedder_agent import EmbedderAgent
from agents.retriever_agent import RetrieverAgent
from agents.qa_agent import QAAgent
from langgraph.graph import StateGraph

ModuleNotFoundError: No module named 'utils'

## Step 1: Load and Chunk PDF

In [ ]:
file_path = f"{base_path}/data/sample_docs/semaglutide_review.pdf"
raw_text = extract_text_from_pdf(file_path)
cleaned = clean_text(raw_text)
chunks = chunk_text(cleaned)


## Step 2: Initialize Agents

In [8]:
import os

print(f"Current base_path: {base_path}")
print(f"Contents of {base_path}:")
try:
    for item in os.listdir(base_path):
        print(item)
except FileNotFoundError:
    print(f"Error: Directory not found at {base_path}")

Current base_path: /content/drive/MyDrive/PharmaAGen
Contents of /content/drive/MyDrive/PharmaAGen:
Error: Directory not found at /content/drive/MyDrive/PharmaAGen


In [ ]:
embedder = EmbedderAgent()
retriever = RetrieverAgent(chunks, embedder)
qa_agent = QAAgent()


## Step 3: Build LangGraph

In [ ]:
builder = StateGraph()

builder.add_node("embedder", embedder.run)
builder.add_node("retriever", retriever.run)
builder.add_node("qa", qa_agent.run)

builder.set_entry_point("retriever")
builder.add_edge("retriever", "qa")

graph = builder.compile()


## Step 4: Query the Graph

In [ ]:
query = "What is the half-life of Semaglutide?"
response = graph.invoke({"query": query})
print(response)
